$${\bf Восстановление\spaceполя\spaceна\spaceзаданной\spaceповерхности\spaceпо\spaceизмереням\spaceдальнего\spaceполя}$$

Пусть у нас имеются измерения поля на некоторой поверхности $\Sigma_M$, а также есть поверхность объема, содержащего внутри себя антенну, $\Sigma_R$. В этой области имеются неокторые источники токов $J$, $M$, по теорема эквивалентности, поле на поверхности $\Sigma_R$:$$E(J,M)=E(J_{eq}, M_{eq})$$ а вне ее $$E(J_{eq}, M_{eq})= 0$$
Удалим антенну и все источники, тогда задача относительно эквивалентных токов примет следующий вид:

Уравнение на поверхности, где произведены измерения $$ n\times E(r)= n\times[-\eta_0 L(J_{eq}, r) + K(M_{eq}, r)], r\in\Sigma_M $$

Уравнение на поверхности $\Sigma_R$ : $$ n\times[-\eta_0L(J_{eq},r)+K(M_{eq},r)]=-\frac{1}{2}M_{eq}(r), r\in \Sigma_R$$

где $L(\cdot)$ и $M(\cdot)$ соответствующие операторы интегрального представления электрического поля:
$$ L(J_{eq}, r)=\int_\Sigma \{\nabla \nabla'[J_{eq}G(r,r')]+k^2 J_{eq}G(r,r')\}dS' $$
$$ K(M_{eq}, r)=\int_\Sigma\{M_{eq}\times\nabla'G(r,r')\}dS'$$



Используя базисные функции RWG - $f_n$, искомые токи представим в виде $J_{eq} = \sum_{n=1}^N C_n^J f_n$ и $M_{eq} = \sum_{n=1}^N C_n^M f_n$. Применяя метод Галеркина, получим систему уравнений:
$$\begin{bmatrix}
-L_{MR}&\space\space -K_{MR} \\
-L_{RR}&\space\space K_{RR}+0.5I_{RR}
\end{bmatrix} 
\begin{bmatrix}
C^J\\
C^M
\end{bmatrix}=
\begin{bmatrix}
E_m\\
0
\end{bmatrix}
\hspace{3cm} [1]$$
где $E_m$ измерения дальнего поля, индексы блоков означают проекции операторов на соответствующие пространства: SR - RWG на пространство измерений, RR - RWG на пространство тестовых функций

$${\bf Постановка\spaceзадачи}$$

Имеется измерения дальнего поля антенны на сфере радиусом 1 км на частоте $0.7$ GHz. Требуется восстановить поле, излучаемое данной антенной на сфере радиусом 1 м.

Для дискретизации поставленной задачи используется BEMPP (https://bempp.com/handbook/api/boundary_operators.html). Здесь есть все необходимые операторы и базисные функции.
Обратим внимание, что количество измерений не совпадает с количеством ребер на сетке, заданной на поверхности, где требуется восстановление. Тогда для нахождения обратной матрицы системы воспользуемся методом наименьших квадратов, а для улучшения обусловленности матрицы - регуляризацией Тихонова.
Полученные коэффициенты - коэффициенты разложения искомых токов по базисным функциям: $J_{eq_n} = \sum_n C_n f_n$, следовательно восстановить поле можно по формуле: $ E_t = M_{eq}\times n$

(https://bempp-cl.readthedocs.io/en/latest/docs/bempp/api/)

Для начала работы установите следующие пакеты:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import bempp.api
import matplotlib as mpl
import plotly.figure_factory as ff
import plotly.graph_objects as go
import pandas as pd

Понадобятся следющие константы:

In [ ]:
frequency = 0.7 * 10**9 
vacuum_permittivity = 8.854187817E-12
vacuum_permeability = 4 * np.pi * 1E-7
k_ext = 2 * np.pi * frequency * np.sqrt(vacuum_permittivity * vacuum_permeability)
wave_length = 2 * np.pi / k_ext
coef = np.sqrt(vacuum_permeability / vacuum_permittivity)

Для составления блоков матрицы системы $[1]$ понадобится вспомогательная функция, вычсиляющая блок матрицы, соответствующий потенциальному оператору, по сеточной функции:

In [ ]:
#  pot_op - потенциальный оператор bempp
#  field_mat - матрица, вычсиляемая путем применения оператора к сеточной функции
def get_mat(pot_op):
    space = pot_op.space
    field_mat = []
    for i in range(space.global_dof_count):
        coef = np.zeros(space.global_dof_count)
        coef[i] = 1
        field = pot_op * bempp.api.GridFunction(space, coefficients=coef)
        field_vec = field.flatten(order='F')
        field_mat.append(field_vec[:, None])
        
    field_mat = np.hstack(field_mat)
    
    return field_mat

$${\bf План\space выполнения}$$
Предлагается написать метод, выполняющий восстановление поля на поверхности <span style="color:green">def</span> <span style="color:blue">current_reconstruction</span>(*) $[2]$, принимающий на вход:
1. файл с сеткой, файл из измерениями дальнего поля
2. волновое число
3. параметр для регуляризации,

а на выходе:
1. координаты центров элементов сетки, на котрой произведено восстановление
2. компоненты нормали
3. реальная часть компонент поля Е
4. мнимая часть компонент поля Е


Для удобства будем использовать шаблон класса, в котором необходимо дописать некотрые методы:

In [19]:
# Bempp import boundary conditions
from bempp.api.operators.potential import maxwell as maxwell_potential
from bempp.api.operators.boundary import maxwell as maxwell_boundary


# Шаблон класса и его методы
class Current_reconstruction_class:
    def __init__(self, wave_number):
        self.wave_number = wave_number
        
    #Определение пространств базисных и тестовых функций на сетке, где требуется восстановление:
    def set_reconstruction_grid(self, grid):
        self.reconstr_grid = grid
        self.reconstr_RWG = bempp.api.function_space(self.reconstr_grid, "RWG", 0) # div_space
        self.reconstr_SNC = bempp.api.function_space(self.reconstr_grid, "SNC", 0) # curl_space
        
    #Определение начальных данных: значения известного дальнего поля и точек, в которых оно было измерено:
    def set_measured_field(self, points, electric_field):
        self.measur_efield = electric_field
        self.measur_points = points 
        
    #Cборка правой части системы:
    def assemble_rhs(self):
        RHS = np.concatenate([
            self.measur_efield.flatten(order='F'), 
            np.zeros(self.reconstr_SNC.global_dof_count)
        ])
        self.rhs = RHS
        
    #Сборка левой части системы $[1]$. Допишите функцию с помощью операторов Bempp:
    def assemble_matrix(self, correction=True):
        # Eto bolshaya setka
        L_SR = maxwell_potential.electric_field(self.reconstr_RWG, 
                                      points=self.measur_points, 
                                      wavenumber=self.wave_number, 
                                      parameters=None, assembler='dense', device_interface=None, precision=None)
        K_SR = maxwell_potential.magnetic_field(self.reconstr_RWG,
                                      points=self.measur_points,
                                      wavenumber=self.wave_number,
                                      parameters=None, assembler='dense', device_interface=None, precision=None)
        L_SR_mat = get_mat(L_SR) # Убрали волновое число для проверки
        K_SR_mat = get_mat(K_SR)
        
        # Eto malenkaya setka, boundary_operator
        L_RR = maxwell_boundary.electric_field(self.reconstr_RWG,
                                               self.reconstr_RWG,
                                               self.reconstr_SNC,
                                               wavenumber=self.wave_number,
                                               parameters=None, assembler='dense', device_interface=None, precision=None).weak_form()
        K_RR = maxwell_boundary.magnetic_field(self.reconstr_RWG, 
                                               self.reconstr_RWG,
                                               self.reconstr_SNC,
                                               wavenumber=self.wave_number,
                                               parameters=None, assembler='dense', device_interface=None, precision=None).weak_form()
        L_RR_mat = L_RR.A
        K_RR_mat = K_RR.A
        I = bempp.api.operators.boundary.sparse.identity(self.reconstr_RWG, self.reconstr_RWG, self.reconstr_SNC).weak_form().A
        K_RR_mat += 0.5*I
        
        self.res_mat = coef * np.hstack((np.vstack((L_SR_mat,L_RR_mat)),np.vstack((K_SR_mat,K_RR_mat))))
        
    #Нахождение решения системы методом наименьших квадратов с регуляризацией Тихонова. Допишите метод.
    def reconstruct_currents(self, method="Least Squares", Tikhonov_reg=0.0,log=0):
        n = self.res_mat.shape[1]
        reg_matrix = np.vstack((self.res_mat, Tikhonov_reg * np.eye(n)))
        reg_b = np.hstack((self.rhs, np.zeros(n)))
        x = np.linalg.lstsq(reg_matrix, reg_b,rcond=None)
        return x[0]
    
    #Вычисление тангенциальных составляющих полей. Функция вовзращает центры элементов, нормали,E_t, H_t.
    #Допишите метод, используя методы класса GridFunction из Bempp:
    def evaluate_tangential_trace_on_element_centers(self):
        coeff = np.sqrt(vacuum_permeability / vacuum_permittivity)
        centers = self.reconstr_grid.centroids
        normals = self.reconstr_grid.normals
        M_trace = 0
        E_trace = 0
        return centers, normals, E_trace, M_trace
    
    def sphere_img(self,E_t):
        dp0_space = bempp.api.function_space(self.reconstr_grid, "DP", 0)
        E_finale = np.asarray([np.sqrt(e[0]*e[0]+e[1]*e[1]+e[2]*e[2]) for e in E_t])
        reference_fun = bempp.api.GridFunction(dp0_space, coefficients=abs(E_finale))
        self.FINALE = E_finale
        reference_fun.plot()

    #Метод, использующий внутри себя все описанные выше функции (детали см. в План выполнения).
    #В качестве параметра регуляризации возьмите 0.001. Допишите метод: 
    def current_reconstruction(self,grid_filename, data_filename, result_filename, k_ext, reg_parameter=1e-3):
            # Load grid (outer)
            print("1 : ", grid_filename, ' ', type(grid_filename))
            grid = bempp.api.import_grid(grid_filename)
            print("TUTA")
            # Get values from outer sphere
            df = pd.read_csv(data_filename,sep=' ', header=None)
            b = df.iloc[:,:3]
            outer_points = b.values
            outer_points.shape
            outer_points = outer_points.T
            outer_values = df.iloc[:,3:6].values + 1j*df.iloc[:,6:].values
            outer_values = outer_values.T

            # Set grid
            self.set_reconstruction_grid(grid)
            print(f'STEP GRID: PASSED')
            # Set field
            self.set_measured_field(outer_points,outer_values)
            print(f'STEP FIELD: PASSED') 
            # Construct matrix
            self.assemble_matrix()
            print(f'STEP CONSTRUCT MATRIX: PASSED') 
            # Construct RHS
            self.assemble_rhs()
            print(f'STEP CONSTRUCT RHS: PASSED')

            # Reconstruct currents
            result = self.reconstruct_currents(Tikhonov_reg=reg_parameter)
            print('L2-norm:\t',np.linalg.norm(self.res_mat.dot(result) - self.rhs))
            print(f'STEP CALCULATE NORM: PASSED')

            # Split result vector into Cj Cm
            C_J = result[:result.shape[0]//2]
            C_M = result[result.shape[0]//2:]
            print(f'STEP FINDING COEFFS: PASSED')

            # Finding J_eq, M_eq
            J_eq = bempp.api.GridFunction(self.reconstr_RWG,dual_space=self.reconstr_SNC,coefficients=C_J)
            M_eq = bempp.api.GridFunction(self.reconstr_RWG,dual_space=self.reconstr_SNC,coefficients=C_M)
            print(f'SETTING J_EQ,M_EQ: PASSED')

            # Finding E_t
            normals = self.reconstr_grid.normals
            M_eq_centers = M_eq.evaluate_on_element_centers().T 
            E_t = np.cross(normals,M_eq_centers)
            print(f'FINDING E_T: PASSED')
            # Graph img
            self.sphere_img(E_t)

In [20]:
# Основной блок, сфера
GRID_PATH = 'Sphere_1m/Sphere_1m_1269edges.msh'
DATA_PATH = './Sphere_1m/Ef_Port1_0_7GHz_Sphere_1km.csv'
SPHERE = Current_reconstruction_class(k_ext)
SPHERE.current_reconstruction(GRID_PATH,DATA_PATH,"dota.csv",k_ext)

1 :  Sphere_1m/Sphere_1m_1269edges.msh   <class 'str'>

TUTA
STEP GRID: PASSED
STEP FIELD: PASSED
STEP CONSTRUCT MATRIX: PASSED
STEP CONSTRUCT RHS: PASSED


$$\bf Общая\spaceструктура \space работы$$

Мы детально разобрали, какие методы понадобятся для решения поставленной задачи. 
Сетка, на которой требуется восстановить токи - <span style="color:green">"test_data/test_data/Sphere_R1_1269edges.msh"</span>, измерения вы найдете в файле - <span style="color:green">"test_data/Ef_Port1_0_7GHz_Sphere_1km.csv"</span>, и референсные значения - <span style="color:green">"test_data/Ef_Port1_0_7GHz_Sphere_1m.csv"</span>. 
Итак, подытожим план выполнения практического задания:
1. С помощью метода Галеркина составить систему уравнений для восстановления эквивалентных токов $J_{eq_n}, M_{eq}$ по известным измерениям $E_m$
2. Используя операторы и пространства в Bempp (https://bempp.com/handbook/api/boundary_operators.html), собрать левую и правую части системы: <span style="color:blue">assemble_matrix</span> , <span style="color:blue">assemble_rhs</span>
3. Найти обратную матрицу системы с помощью МНК и регуляризации Тихонова: <span style="color:blue">reconstruct_currents</span>.
4. По найденным коэффициентам разложения токов найти тангенциальную составляющую поля $E$ на заданной поверхности: <span style="color:blue">current_reconstruction</span>
6. Сравнить полученные результаты с референсом

In [ ]:
a = Current_reconstruction_class(k_ext)
a.assemble_matrix()